<a href="https://colab.research.google.com/github/danimorenod/Entregas/blob/main/Proyecto_Final_Daniela_Moreno_D%C3%A1vila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run streamlit app from a Google Colab Notebook
> Created by [Manuel Romero](https://twitter.com/mrm8488)

In [31]:
!pip install -q streamlit

In [32]:
!pip install streamlit yagmail


## Create a streamlit app example


In [33]:

%%writefile app.py
import csv
from datetime import datetime
import yagmail
import streamlit as st
from random import choice

# Clase: BirthdayManager
class BirthdayManager:
    def __init__(self, data_file="birthdays.csv"):
        self.data_file = data_file
        self.birthdays = []
        self.load_birthdays()

    def load_birthdays(self):
        """Carga los cumpleaños desde el archivo CSV"""
        try:
            with open(self.data_file, mode='r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                self.birthdays = list(reader)
        except FileNotFoundError:
            self.birthdays = []

    def save_birthdays(self):
        """Guarda los cumpleaños en el archivo CSV"""
        with open(self.data_file, mode='w', newline='', encoding='utf-8') as file:
            fieldnames = ['name', 'email', 'birth_date']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(self.birthdays)

    def add_birthday(self, name, email, birth_date):
        """Agrega un nuevo cumpleaños a la lista"""
        self.birthdays.append({
            "name": name.strip(),
            "email": email.strip(),
            "birth_date": birth_date
        })
        self.save_birthdays()

    def delete_birthday(self, name):
        """Elimina un cumpleaños de la lista por nombre"""
        normalized_name = name.strip().lower()
        filtered_birthdays = [
            b for b in self.birthdays if b['name'].strip().lower() != normalized_name
        ]
        if len(filtered_birthdays) < len(self.birthdays):  # Si se eliminó algo
            self.birthdays = filtered_birthdays
            self.save_birthdays()
            return True
        return False

    def get_upcoming_birthdays(self):
        """Devuelve una lista de los cumpleaños próximos"""
        today = datetime.now().date()
        upcoming = []
        for person in self.birthdays:
            birth_date = datetime.strptime(person['birth_date'], '%Y-%m-%d').date()
            next_birthday = birth_date.replace(year=today.year)
            if next_birthday < today:
                next_birthday = next_birthday.replace(year=today.year + 1)
            days_until = (next_birthday - today).days
            upcoming.append({
                "name": person['name'],
                "email": person['email'],
                "birth_date": person['birth_date'],
                "days_until": days_until
            })
        return sorted(upcoming, key=lambda x: x['days_until'])


# Clase: EmailSender
class EmailSender:
    def __init__(self, sender_email, password):
        self.sender_email = sender_email
        self.password = password
        try:
            self.yag = yagmail.SMTP(user=self.sender_email, password=self.password)
        except Exception as e:
            raise Exception(f"Error al inicializar la conexión de correo: {e}")

    def send_email(self, receiver_email, subject, message):
        """Envía un correo electrónico utilizando Yagmail"""
        try:
            self.yag.send(
                to=receiver_email,
                subject=subject,
                contents=message
            )
        except Exception as e:
            raise Exception(f"Error al enviar el correo: {e}")

    def get_random_message(self, messages_file="messages.txt"):
        """Obtiene un mensaje aleatorio del archivo"""
        try:
            with open(messages_file, 'r', encoding='utf-8') as file:
                messages = file.readlines()
            return choice(messages).strip()
        except FileNotFoundError:
            return "¡Feliz cumpleaños! 🥳 pasatela chido."


# Aplicación con Streamlit
# Configuración inicial
birthday_manager = BirthdayManager()
email_sender = EmailSender(
    sender_email="daniela.moreno0248@alumnos.udg.mx",
    password="uxjr ccwi dwzv hjvd"
)

st.title("Gestor de Cumpleaños")

# Registro de cumpleaños
st.header("Registrar un Cumpleaños")
name = st.text_input("Nombre")
email = st.text_input("Correo Electrónico")
birth_date = st.date_input(
    "Fecha de Nacimiento",
    min_value=datetime(1900, 1, 1),
    max_value=datetime.now()
)
if st.button("Agregar Cumpleaños"):
    if name and email and birth_date:
        birthday_manager.add_birthday(name, email, birth_date.strftime('%Y-%m-%d'))
        st.success("Cumpleaños registrado correctamente.")
    else:
        st.error("Por favor completa todos los campos.")

# Lista de cumpleaños próximos
st.header("Próximos Cumpleaños")
upcoming = birthday_manager.get_upcoming_birthdays()
if upcoming:
    for person in upcoming:
        st.write(f"{person['name']} - {person['days_until']} días ({person['birth_date']})")
else:
    st.write("No hay cumpleaños próximos registrados.")

# Enviar felicitaciones
st.header("Enviar Felicitaciones")
if st.button("Enviar Felicitaciones"):
    today = datetime.now().date()
    sent_any = False
    for person in birthday_manager.birthdays:
        birth_date = datetime.strptime(person['birth_date'], '%Y-%m-%d').date()
        if birth_date.month == today.month and birth_date.day == today.day:
            message = email_sender.get_random_message()
            try:
                email_sender.send_email(
                    receiver_email=person['email'],
                    subject="Feliz Cumpleaños 🎂🥳🎉🙌",
                    message=message
                )
                st.success(f"Felicitación enviada a {person['name']}")
                sent_any = True
            except Exception as e:
                st.error(f"Error al enviar correo a {person['name']}: {e}")
    if not sent_any:
        st.info("No hay cumpleaños hoy para enviar felicitaciones.")

# Eliminar cumpleaños
st.header("Eliminar un Cumpleaños")
name_to_delete = st.text_input("Nombre del contacto a eliminar")
if st.button("Eliminar Cumpleaños"):
    if name_to_delete:
        success = birthday_manager.delete_birthday(name_to_delete.strip())
        if success:
            st.success(f"El cumpleaños de {name_to_delete} ha sido eliminado.")
        else:
            st.error(f"No se encontró ningún contacto con el nombre {name_to_delete}.")
    else:
        st.error("Por favor, ingresa un nombre para eliminar.")


Overwriting app.py


## Install localtunnel

In [34]:
!npm install localtunnel

⠙⠹⠸⠼
up to date, audited 23 packages in 807ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠼

## Run streamlit in background

In [35]:
!streamlit run /content/app.py &>/content/logs.txt &

## Expose the port 8501
Then just click in the `url` showed.

A `log.txt`file will be created.

In [36]:
!npx localtunnel --port 8501

⠙your url is: https://itchy-days-bake.loca.lt
^C


[![ko-fi](https://www.ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Y8Y3VYYE)